In [1]:
using LinearAlgebra, NBInclude, IterativeSolvers, FunctionOperators

In [2]:
@nbinclude("helper_functions.ipynb")

In [3]:
Random.seed!(0);

## Fancy MatrixIRLS for matrix completion (PCA)

_**Note:** Fancy = The weighted least squares step is optimized by computing it in a lower dimensional space using the Sherman-Morrison-Woodbury form._

### Sources
 - Preprint paper by Christian Kümmerle & Claudio Verdun: https://arxiv.org/pdf/0912.3599.pdf
 - GitHub repo of the preprint paper: https://github.com/ckuemmerle/MatrixIRLS

### Algorithm
 - **Input:** Sampling operator $\Phi$, observations $\mathbf{y} \in \mathbb{C}^m$, rank estimate $\tilde{r}$, iteration number $N$.
 - Initialize $k=0, \epsilon_0 = \infty, W^{(0)} = Id.$
 - **for $k=1$ to $N$ do**
    1. **Solve weighted least squares:** Use a *conjugate gradient method* to solve $$\mathbf{X}^{(k)} = argmin \langle \mathbf{X}, W^{(k-1)}(\mathbf{X}) \rangle \text{ subject to } \Phi(\mathbf{X}) = \mathbf{y}.$$
    2. **Update smoothing:** Compute $\tilde{r}+1$-th singular value of $\mathbf{X}^{(k)}$ to update $$\epsilon_k = min\left(\epsilon_{k-1}, \sigma_{\tilde{r}+1}(\mathbf{X}^{(k)})\right).$$
    3. **Update weight operator:** For $r_k := \left\vert\{i \in [d] : \sigma_i(\mathbf{X}^{(k)}) > \epsilon_k\}\right\vert$, compute the first $r_k$ singular values $\sigma_i^{(k)} := \sigma_i^{(k)}(\mathbf{X}^{(k)})$ and matrices $\mathbf{U}^{(k)} \in \mathbb{C}^{d_1 \times r_k}$ and $\mathbf{V}^{(k)} \in \mathbb{C}^{d_2 \times r_k}$ with leading $r_k$ left/right singular vectors of $\mathbf{X}^{(k)}$ to update $W^{(k)}$: $$W^{(k)}(\mathbf{Z}) = \mathbf{U}^{(k)} \left[ \mathbf{H}_k \circ (\mathbf{U}^{(k)*} \mathbf{Z} \mathbf{V}^{(k)})\right]\mathbf{V}^{(k)*},$$ where $\circ$ denotes the entrywise product of two matrices, and $\mathbf{H}_k \in \mathbb{C}^{d_1 \times d_2}$ matrix defined as $$(\mathbf{H}_k)_{ij} := \left(\max(\sigma_i^{(k)}, \epsilon^{(k)}\max(\sigma_j^{(k)}, \epsilon^{(k)}\right)^{-1} : \forall i \in [d_1] \text{ and } \forall j \in [d_2].$$
 - **end**
 - **Output**: $\mathbf{X}^{(k)}$

### Technical details

#### Optimization of the computation of the weighted least squares
In order to reduce the computational complexity, the weighted least squares step in the algorithm above is computed in a lower dimensional space.
 - For a given rank rk, we recall that the best rank-$r_k$ approximation of a matrix $X(k)$ can be written such that $$\mathcal{T}_{r_k}(\mathbf{X}^{(k)}) := \underset{\mathbf{Z}:rank(\mathbf{Z}) < r_k}{\arg\min} \Vert\mathbf{Z} - \mathbf{Z}^{(k)} \Vert = \mathbf{U}^{(k)} \boldsymbol{\Sigma}^{(k)} \mathbf{V}^{(k)*},$$ where $\Vert \cdot \Vert$ can be any unitary invarian norm.
 - Let now $$T_k := T_{\mathcal{T}_{r_k}(\mathbf{X}^{(k)})}\mathcal{M}_{r_k} :=  \left\{\mathbf{U}^{(k)} \Gamma_1 \mathbf{V}^{(k)*} + \mathbf{U}^{(k)} \Gamma_2 (\mathbf{I} - \mathbf{V}^{(k)} \mathbf{V}^{(k)*}) + (\mathbf{I} - \mathbf{U}^{(k)} \mathbf{U}^{(k)*}) \Gamma_3 \mathbf{V}^{(k)*} : \\ \Gamma_1 \in \mathbb{C}^{r_k \times r_k}, \Gamma_2 \in \mathbb{C}^{r_k \times d_2}, \Gamma_2 \mathbf{V}^{(k)} = \mathbf{0}, \Gamma_3 \in \mathbb{C}^{d_1 \times r_k}, \mathbf{U}^{(k)*}\Gamma_3 = \mathbf{0}\right\}$$ be the tangent space of the manifold of rank-$r_k$ matrices $\mathcal{M}_{r_k}$ of dimension $(d_1 \times d_2)$ at $\mathcal{T}_{r_k}(\mathbf{X}^{(k)})$.
 - For practical considerations we need to introduce vector space $$S_k := \left\{ \gamma = (\gamma_1^T, \gamma_2^T, \gamma_3^T)^T \in \mathbb{C}^{d_1 + d_2 + r_k} : \Gamma_1 = (\gamma_1)_{mat} \in \mathbb{C}^{r_k \times r_k}, \Gamma_2 = (\gamma_2)_{mat} \in \mathbb{C}^{r_k \times d_2}, \Gamma_2 \mathbf{V}^{(k)} = \mathbf{0}, \Gamma_3 = (\gamma_3)_{mat} \in \mathbb{C}^{d_1 \times r_k}, \mathbf{U}^{(k)*}\Gamma_3 = \mathbf{0}\right\} \subset \mathbb{C}^{r_k(d_1 + d_2 + r_k)},$$ where $mat$ is the matricization operator of appropriate dimension that stacks column after column according to the desired dimensions.
 - We can now identify a structure in $W^{(k)}$ that enables us to write it more compactly: Let $P_{T_k}: S_k \rightarrow T_k$ be the parametrization operator such that $$P_{T_k}(\gamma) := \mathbf{U}^{(k)} \Gamma_1 \mathbf{V}^{(k)*} + \mathbf{U}^{(k)} \Gamma_2 (\mathbf{I} - \mathbf{V}^{(k)} \mathbf{V}^{(k)*}) + (\mathbf{I} - \mathbf{U}^{(k)} \mathbf{U}^{(k)*}) \Gamma_3 \mathbf{V}^{(k)*} : \gamma \in S_k$$.
 - As we know that $\Gamma_2 \mathbf{V}^{(k)} = \mathbf{0}$, and $\mathbf{U}^{(k)*}\Gamma_3 = \mathbf{0}$, we can simplify the parametrization operator: $$P_{T_k}(\gamma) = \mathbf{U}^{(k)} \Gamma_1 \mathbf{V}^{(k)*} + \mathbf{U}^{(k)} \Gamma_2 + \Gamma_3 \mathbf{V}^{(k)*}$$
 - Its adjoint operator $$P_{T_k}^*(\mathbf{Z}) = \left((\mathbf{U}^{(k)*} \mathbf{Z} \mathbf{V}^{(k)})_{vec}^T, (\mathbf{U}^{(k)*} Z (\mathbf{I} - \mathbf{V}^{(k)} \mathbf{V}^{(k)*}))_{vec}^T, ((\mathbf{I} - \mathbf{U}^{(k)} \mathbf{U}^{(k)*}) \mathbf{Z} \mathbf{V}^{(k)*})_{vec}^T\right)^T : \mathbf{Z} \in \mathbb{C}^{d_1 \times d_2}$$. 

 - Let us divide matrix $\mathbf{H}^{(k)}$ into four blocks $$\mathbf{H}^{(k)} = \begin{bmatrix}\mathbf{H}^{(k)}_{1,1} & \mathbf{H}^{(k)}_{1,2} \\ \mathbf{H}^{(k)}_{2,1} & \epsilon_k^{-2} \mathbf{1}\end{bmatrix},$$ such that $\mathbf{H}^{(k)}_{1,1} \in \mathbb{C}^{r_k \times r_k}$ with $$(\mathbf{H}^{(k)}_{1,1})_{i,j} = \left(\sigma_i \sigma_j \right)^{-1},$$ and define diagonal matrix $\mathbf{D}^{(k)} \in \mathbb{C}^{r_k \times r_k}$ as $$\mathbf{D}_{i,i}^{(k)} := \left(\sigma_i^{(k)} \epsilon_k \right)^{-1}.$$
 - As all columns of $\mathbf{H}^{(k)}_{1,2}$ / all rows of $\mathbf{H}^{(k)}_{2,1}$ has the same values as the diagonal of $\mathbf{D}^{(k)}$, we can replace the element-wise multiplication by block $\mathbf{H}^{(k)}_{1,2}$ / $\mathbf{H}^{(k)}_{2,1}$ with left / right matrix multiplication by $\mathbf{D}^{(k)}$: $$\begin{equation*}
\begin{split}
W^{(k)}(\mathbf{Z}) &= \mathbf{U}_k \left[\mathbf{H}_k \circ (\mathbf{U}_k^{*} \mathbf{Z} \mathbf{V}_k)\right] \mathbf{V}_k^{*} \\
&=\begin{bmatrix} 
    \mathbf{U}^{(k)} & \mathbf{U}_{\perp}^{(k)}
\end{bmatrix}
\left(\mathbf{H}_k
\circ
\begin{bmatrix}
\mathbf{U}^{(k)*} \mathbf{Z} \mathbf{V}^{(k)} &  \mathbf{U}^{(k)*} \mathbf{Z} \mathbf{V}_{\perp}^{(k)} \\
\mathbf{U}_{\perp}^{(k)*} \mathbf{Z} \mathbf{V}^{(k)} & \mathbf{U}_{\perp}^{(k)*} \mathbf{Z} \mathbf{V}_{\perp}^{(k)} 
\end{bmatrix}
\right)
\begin{bmatrix} 
    \mathbf{V}^{(k)*} \\ \mathbf{V}_{\perp}^{(k)*}
\end{bmatrix} \\
&= \begin{bmatrix} 
    \mathbf{U}^{(k)} & \mathbf{U}_{\perp}^{(k)}
\end{bmatrix}
\left(
\begin{bmatrix}
\mathbf{H}^{(k)} &  \mathbf{H}_{1,2}^{(k)} \\
\mathbf{H}_{2,1}^{(k)} & \epsilon_k^{-2} \mathbf{1}
\end{bmatrix} 
\circ 
\begin{bmatrix}
\mathbf{U}^{(k)*} \mathbf{Z} \mathbf{V}^{(k)} &  \mathbf{U}^{(k)*} \mathbf{Z} \mathbf{V}_{\perp}^{(k)} \\
\mathbf{U}_{\perp}^{(k)*} \mathbf{Z} \mathbf{V}^{(k)} & \mathbf{U}_{\perp}^{(k)*} \mathbf{Z} \mathbf{V}_{\perp}^{(k)} 
\end{bmatrix}
\right)
\begin{bmatrix} 
    \mathbf{V}^{(k)*} \\ \mathbf{V}_{\perp}^{(k)*}
\end{bmatrix}  \\
&=
\begin{bmatrix} 
    \mathbf{U}^{(k)} & \mathbf{U}_{\perp}^{(k)}
\end{bmatrix}
\begin{bmatrix}
\mathbf{H}^{(k)} \circ \left(\mathbf{U}^{(k)*} \mathbf{Z} \mathbf{V}^{(k)}\right) &  \mathbf{D}^{(k)} \mathbf{U}^{(k)*} \mathbf{Z} \mathbf{V}_{\perp}^{(k)} \\
\mathbf{U}_{\perp}^{(k)*} \mathbf{Z} \mathbf{V}^{(k)}\mathbf{D}^{(k)}  & \epsilon_k^{-2} \mathbf{U}_{\perp}^{(k)*} \mathbf{Z} \mathbf{V}_{\perp}^{(k)} 
\end{bmatrix}
\begin{bmatrix} 
    \mathbf{V}^{(k)*} \\ \mathbf{V}_{\perp}^{(k)*}
\end{bmatrix},
\end{split}
\end{equation*}$$
 - Finally, we define $\mathbf{D}_{S_k} \in \mathbb{C}^{r_k(d_1 + d_2 + r_k) \times r_k(d_1 + d_2 + r_k)}$ as a diagonal matrix with diagonal entries that are equal to entries of $\mathbf{H}^{(k)}_{1,1}$ or $\mathbf{D}^{(k)}$: $$\mathbf{D}_{S_k} = \begin{bmatrix} diag\left((\mathbf{H}^{(k)}_{1,1})_{vec}\right) & & 0 \\ & \mathbf{D}^{(k)} \otimes \mathbf{I}_{(d_1 \times d_1)} & \\ 0 & & \mathbf{I}_{(d_2 \times d_2)} \otimes \mathbf{D}^{(k)}\end{bmatrix},$$ where $diag$ transforms a vector to a diagonal matrix, $\otimes$ denotes the Kronecker-product, and $\mathbf{I}_{(d_1 \times d_1)}, \mathbf{I}_{(d_2 \times d_2)}$ are identity matrices of size $(d_1 \times d_1)$ and $(d_2 \times d_2)$.
   - It's easy to see that $diag\left((\mathbf{H}^{(k)}_{1,1})_{vec}\right) (\mathbf{M})_{vec} = \left(\mathbf{H}^{(k)}_{1,1} \circ \mathbf{M}\right)_{vec}$.
   - Kronecker-product is needed to transform diagonal-matrix&ndash;matrix multiplication to diagonal-matrix&ndash;vectorized-matrix multiplication. E.g. $\mathbf{AB} = (\mathbf{I} \otimes \mathbf{A})\mathbf{B}$ when $\mathbf{A}$ is a diagonal matrix, and $\mathbf{AB} = (\mathbf{B}^* \otimes \mathbf{I})\mathbf{A}$ when $\mathbf{B}$ is a diagonal matrix.

 - Using the definitions above, we can re-formulate $\mathbf{W}^{(k)}$ as $$\mathbf{W}^{(k)} = P_{T_k} \left(\mathbf{D}_{S_k} - \epsilon_k^{-2} \mathbf{I}_{S_k}\right)P_{T_k}^* + \epsilon_k^{-2} \mathbf{I},$$ and we can summarize the optimized implementation of the conjugate gradient step for matrix completion:
   1. Calculate $P^*_{T_k} \Phi^*(\mathbf{y}) \in S_k$
   2. Solve $\left(\frac{\epsilon_k^2 \mathbf{I}_{S_k}}{\mathbf{D}_{S_k}^{-1} - \epsilon_k^2 \mathbf{I}_{S_k}} + P^*_{T_k} \Phi^* \Phi P_{T_k}\right)\gamma_k = P^*_{T_k} \Phi^*(\mathbf{y})$ for $\gamma_k \in S_k$ by conjugate gradient method.
   3. Calculate residual $\mathbf{r}_k := y - \Phi P_{T_k} \gamma_k \in \mathbb{C}^m$
   4. Calculate $\tilde{\gamma}_k = \left(\frac{\mathbf{D}_{S_k}^{-1}}{\mathbf{D}_{S_k}^{-1} - \epsilon_k^2 \mathbf{I}_{S_k}}\right)\gamma_k - P^*_{T_k} \Phi^*(r_k) \in S_k$.
   5. Obtain an implicit representation of the new iterate $\mathbf{X}^{(k+1)} \in \mathbb{C}^{d_1 \times d_2}$ such that $\mathbf{X}^{(k+1)} = \Phi^*(\mathbf{r}_k) + P_{T_k}(\tilde{\gamma}_k)$. 

In [4]:
function fancy_MatrixIRLS_for_PCA(
        Xᴳᵀ::AbstractArray,                     # ground truth for MSE evaluation
        y::AbstractArray,                       # under-sampled data
        Φ::FunctionOperator;                    # sampling operator
        img_size::NTuple = size(Xᴳᵀ),           # size of output matrix
        r̃::Int = 0,                             # rank estimate of solution
        maxIter::Union{Int, Nothing} = nothing, # number of CG iteration steps
        N::Int = 1000,                          # number of iterations
        verbose::Bool = false)                  # print rank and loss value in each iteration
    
    # Initialize variables
    dType = eltype(y)
    d₁, d₂ = img_size
    r̃ == 0 && (r̃ = rank(Xᴳᵀ))
    maxIter = maxIter isa Nothing ? r̃*(r̃+d₁+d₂) : maxIter
    ϵₖ = Inf
    Xᵏ = Φ' * y # Initial guess: fill missing values with zeros
    σ, k = 0, 0 # I just want them to be available outside of the loop
    
    verbose && (table = DebugTableModule.DebugTable(
        ("k", () -> k, 3), ("rank(Xᵏ)", () -> rank(Xᵏ, atol=1e-3), 3),
        ("‖Xᴳᵀ - Xᵏ‖₂", () -> opnorm(Xᴳᵀ - Xᵏ, 2), 3), ("σ₁", () -> σ[1]),
        ("σᵣ₊₁", () -> σ[r̃+1]), ("ϵₖ", () -> ϵₖ)))
    
    while k <= N && ϵₖ > 1e-3
        
        # Find leading rₖ left/right singular vectors of Xᵏ and calculate all singular values
        F = svd(Xᵏ)
        Uᵏ, σ, Vᵏ = F.U[:, 1:r̃], F.S, F.V[:, 1:r̃]
        
        # Print some info
        verbose && printRow(table)
        
        # Step 2.
        ϵₖ = min(ϵₖ, σ[r̃+1])
        
        # Step 3.
        # We skip calculating Wᵏ in favour of the optimized implementation:
        Hᵏ₁₁ = [1 / (σ[i] * σ[j])  for i in 1:r̃, j in 1:r̃]
        Dᵏ = Diagonal([1 / (σ[i] * ϵₖ)  for i in 1:r̃])
        Pᵏ = FunctionOperator{dType}(name="Pᵏ", inDims = (r̃*(r̃+d₁+d₂),), outDims = (d₁, d₂),
            forw = γ -> begin
                    Γ₁ = reshape(γ[1:r̃^2], r̃, r̃)
                    Γ₂ = reshape(γ[r̃^2+1:r̃*(r̃+d₂)], r̃, d₂)
                    Γ₃ = reshape(γ[r̃*(r̃+d₂)+1:r̃*(r̃+d₁+d₂)], d₁, r̃)
                    Uᵏ * Γ₁ * Vᵏ' + Uᵏ * Γ₂ + Γ₃ * Vᵏ'
                end,
            backw = Φᵃy -> begin
                    Γ₁ = Uᵏ' * Φᵃy * Vᵏ
                    Γ₂ = Uᵏ' * Φᵃy * (I - Vᵏ*Vᵏ')
                    Γ₃ = (I - Uᵏ*Uᵏ') * Φᵃy * Vᵏ
                    vcat(vec(Γ₁), vec(Γ₂), vec(Γ₃))
                end)
        I⁽ᵈ¹ˣᵈ¹⁾, I⁽ᵈ²ˣᵈ²⁾ = Diagonal(ones(d₁)), Diagonal(ones(d₂))
        D_Sₖ = Diagonal( vcat( vec(Hᵏ₁₁), diag(kron(Dᵏ, I⁽ᵈ¹ˣᵈ¹⁾)), diag(kron(I⁽ᵈ²ˣᵈ²⁾, Dᵏ)) ) )
        D_Sₖ⁻¹ = I / D_Sₖ
        CG_op = FunctionOperator{dType}(name = "CG_op", inDims = (r̃*(r̃+d₁+d₂),), outDims = (r̃*(r̃+d₁+d₂),),
            forw = γ -> (ϵₖ^2 * I / (D_Sₖ⁻¹ - ϵₖ^2 * I)) * γ + Pᵏ' * Φ' * Φ * Pᵏ * γ)
        
        # Step 1.
        b = Pᵏ' * Φ' * y
        γᵏ = cg(CG_op, b, maxiter = maxIter)
        rᵏ = y - Φ * Pᵏ * γᵏ
        γ̃ₖ = (D_Sₖ⁻¹ / (D_Sₖ⁻¹ - ϵₖ^2 * I)) * γᵏ - Pᵏ' * Φ' * rᵏ
        Xᵏ = Φ' * rᵏ + Pᵏ * γ̃ₖ
        
        k += 1
    end
    
    # Print some info
    verbose && printRow(table, last = true)
    
    return Xᵏ
end

fancy_MatrixIRLS_for_PCA (generic function with 1 method)

# Numerical Experiments

### General parameters

In [5]:
d₁, d₂ = 50, 50     # Matrix dimensions
r = 7               # Desired rank
dType = ComplexF64; # Type of matrix elements

### Generate Model

#### Sampling Mask ($\Phi$)

_**Requirement towards the sampling mask:** It must have at least $r$ non-zero entries in each row and each column._

In [6]:
df = r * (d₁ + d₂ - r) # Number of degrees of freedom of the setting
m = floor(Int, min(1.05 * df, d₁ * d₂))
Φᴹ = generateΦ(d₁, d₂, r, m)
Φ = FunctionOperator{dType}(name = "Φ", inDims = (d₁, d₂), outDims = (d₁, d₂),
    forw = (b,x) -> b .= Φᴹ .* x, backw = (b,x) -> b .= x)
@show r
println("minimum number of non-zero entries in each column: ", Int(minimum(sum(Φᴹ, dims=1))))
println("minimum number of non-zero entries in each column: ", Int(minimum(sum(Φᴹ, dims=2))))

r = 7
minimum number of non-zero entries in each column: 9
minimum number of non-zero entries in each column: 9


### Generate Data

Create a random rank-$r$ matrix $L_0 \in \mathbb{C}^{d_1 \times d_2}$ such that $L_0 = U_0 V_0^*$, where $U_0 \in \mathbb{C}^{d_1 \times r}$ and $V_0 \in \mathbb{C}^{d_2 \times r}$, and then sub-sample this low-rank matrix.

In [7]:
L₀ = generateLowRankComponent_Christian(d₁, d₂, r, dType)
@show size(L₀)
@show rank(L₀)

y = Φ * L₀
@show rank(y);

size(L₀) = (50, 50)
rank(L₀) = 7
rank(y) = 50


### Running The Reconstruction

In [8]:
@time fancy_MatrixIRLS_for_PCA(L₀, y, Φ, verbose = true);

┌─────┬──────────┬─────────────┬──────────┬──────────┬──────────┐
│  k  │ rank(Xᵏ) │ ‖Xᴳᵀ - Xᵏ‖₂ │    σ₁    │   σᵣ₊₁   │    ϵₖ    │
├─────┼──────────┼─────────────┼──────────┼──────────┼──────────┤
│   0 │       50 │      51.429 │   23.947 │   13.997 │      Inf │
│   1 │       50 │      45.687 │   33.877 │   10.724 │   13.997 │
│   2 │       50 │      40.310 │   46.632 │    8.116 │   10.724 │
│   3 │       50 │      36.261 │   54.968 │    6.034 │    8.116 │
│   4 │       50 │      33.008 │   59.068 │    4.641 │    6.034 │
│   5 │       50 │      30.517 │   61.453 │    3.905 │    4.641 │
│   6 │       50 │      28.461 │   63.133 │    3.264 │    3.905 │
│   7 │       50 │      26.237 │   64.420 │    2.435 │    3.264 │
│   8 │       50 │      24.021 │   65.578 │    1.763 │    2.435 │
│   9 │       50 │      22.164 │   66.538 │    1.388 │    1.763 │
│  10 │       50 │      20.958 │   67.265 │    1.080 │    1.388 │
│  11 │       50 │      19.952 │   67.790 │    0.838 │    1.080 │
│  12 │   